In [1]:
# Data processing
import pandas as pd
from surprise import Reader
from surprise import Dataset, SVD, accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import KNNWithMeans

In [2]:
import os
os.getcwd()

'c:\\Users\\Manish Seal\\OneDrive - National University of Singapore\\Documents\\NUS stuff\\y3s2\\DSA4212\\collaborative_filtering\\src'

In [3]:
matrix = pd.read_pickle("../data/item_matrix.pkl")
matrix.head()

userID,0,4,5,7,14,20,31,33,40,46,...,199956,199969,199973,199974,199975,199976,199980,199988,199990,199996
songID,,,,,,,,,,,,,,,,,,,,,
2263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
3785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8063,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
print(matrix.min().min(), matrix.max().max())

1.0 5.0


In [4]:
def get_data(matrix):
    transposed_matrix = matrix.transpose()
    matrix_filled = transposed_matrix.fillna(0)
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(matrix_filled.stack().reset_index(), reader)
    return data

In [7]:
# 5 fold cross-validation
algo = SVD()
cross_validate(algo, get_data(matrix), cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0832  1.0842  1.0845  1.0836  1.0834  1.0838  0.0005  
MAE (testset)     1.0342  1.0349  1.0350  1.0345  1.0343  1.0346  0.0003  
Fit time          40.24   26.78   40.12   49.60   43.90   40.13   7.51    
Test time         6.67    5.17    10.22   9.87    10.18   8.42    2.10    


{'test_rmse': array([1.08315682, 1.0842477 , 1.08448726, 1.08358535, 1.08344028]),
 'test_mae': array([1.03416856, 1.03487175, 1.03501405, 1.03453097, 1.03430595]),
 'fit_time': (40.24217224121094,
  26.77756428718567,
  40.12215805053711,
  49.598525285720825,
  43.90136361122131),
 'test_time': (6.6686952114105225,
  5.168081521987915,
  10.218892812728882,
  9.869235277175903,
  10.179834842681885)}

In [5]:
data = get_data(matrix)
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=0.2)
# We'll use the famous SVD algorithm.
algo = SVD()
# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)
# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 1.0835


1.0834874839067483

In [6]:
sim_options = {
    'user_based': False
}
algo = KNNWithMeans(sim_options=sim_options)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0835


1.0834874839067483

In [2]:
# using fastai here
from fastai.collab import *
from fastai.tabular import *
import seaborn as sns

In [6]:
transposed_matrix = matrix.transpose()
matrix_filled = transposed_matrix.fillna(0)
ratings = matrix_filled.stack().reset_index()

In [7]:
ratings.columns = ['userId', 'songID', 'rating']

In [8]:
data = CollabDataLoaders.from_df(ratings, seed=42, valid_pct=0.2)

In [13]:
data.show_batch()

,userId,songID,rating
0,55079,72309,2.0
1,38545,62954,0.0
2,78785,55240,0.0
3,16691,19299,0.0
4,66802,40712,0.0
5,105654,113954,0.0
6,132358,94535,0.0
7,158011,36561,0.0
8,60053,126757,0.0
9,64516,105433,0.0


AttributeError: 'NoneType' object has no attribute 'columns'

In [19]:
learn = collab_learner(data, n_factors=2, y_range=(0.5, 5.5))

In [20]:
learn.fit_one_cycle(1, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.518308,0.520652,09:10
